## 1. Load NERs extracted previously

In [4]:
import json

with open("../out/prompting_ners_gpt_4.1_mini.json", "r", encoding="utf-8") as fr:
    prompting_ners = json.load(fr)
    
with open("../out/spacy_ners.json", "r", encoding="utf-8") as fr:
    spacy_ners = json.load(fr)

In [4]:
prompt_ner_cnt = 0
spacy_ner_cnt = 0

print("Prompting:")
for key in prompting_ners.keys():
    print(f"\t- {key}: {len(prompting_ners[key])}")
    prompt_ner_cnt += len(prompting_ners[key])

print(f"\nNumber of NERs with Prompting: {prompt_ner_cnt}\n")

print("Spacy:")
for key in spacy_ners.keys():
    print(f"\t- {key}: {len(spacy_ners[key])}")
    spacy_ner_cnt += len(spacy_ners[key])

print(f"\nNumber of NERs with SpaCy: {spacy_ner_cnt}")


Prompting:
	- PER: 192
	- DATE: 118
	- GPE: 24
	- ORG: 22
	- WORK_OF_ART: 66
	- LOC: 30
	- EVENT: 22
	- MONEY: 23
	- LAW: 1

Number of NERs with Prompting: 498

Spacy:
	- PER: 285
	- MISC: 124
	- LOC: 109
	- ORG: 9

Number of NERs with SpaCy: 527


NERs extracted by prompting appears more precise than SpaCy.

In [5]:
for key, values in prompting_ners.items():
    for ner in values:
        found = False
        for spacy_key, prompt_values in spacy_ners.items():
            if ner in prompt_values:
                if key != spacy_key:
                    print(f"'{ner}' with different key: '{key}' in Prompting and '{spacy_key}' in SpaCy")
                break
        else:
            print(f"'{ner}' found in Prompting under key '{key}' but not in SpaCy")

'La Romerilla' found in Prompting under key 'PER' but not in SpaCy
'Francisco García 'el Pupilo'' found in Prompting under key 'PER' but not in SpaCy
'Subirá' with different key: 'PER' in Prompting and 'LOC' in SpaCy
'Manuela de Escamilla' found in Prompting under key 'PER' but not in SpaCy
'Luis de Ulloa y Pereira' found in Prompting under key 'PER' but not in SpaCy
'Tomás de Nájera' found in Prompting under key 'PER' but not in SpaCy
'Príncipe de Astillano' with different key: 'PER' in Prompting and 'LOC' in SpaCy
'Mira' found in Prompting under key 'PER' but not in SpaCy
'princesa de Mélito' found in Prompting under key 'PER' but not in SpaCy
'Catalina de Nicolás y la Rosa' found in Prompting under key 'PER' but not in SpaCy
'Simón de Alcántara' found in Prompting under key 'PER' but not in SpaCy
'Luciana' found in Prompting under key 'PER' but not in SpaCy
'Pérez Pastor' found in Prompting under key 'PER' but not in SpaCy
'Bernarda' with different key: 'PER' in Prompting and 'MISC'

In [6]:
for key, values in spacy_ners.items():
    for ner in values:
        found = False
        for prompt_key, prompt_values in prompting_ners.items():
            if ner in prompt_values:
                if key != prompt_key:
                    print(f"'{ner}' with different key: '{key}' in SpaCy and '{prompt_key}' in Prompting")
                break
        else:
            print(f"'{ner}' found in SpaCy under key '{key}' but not in Prompting")

'Andrómeda' found in SpaCy under key 'PER' but not in Prompting
'Según F. Sáez Raposo' found in SpaCy under key 'PER' but not in Prompting
'Oro' found in SpaCy under key 'PER' but not in Prompting
'Baccio' found in SpaCy under key 'PER' but not in Prompting
'Rey.- ¿Qué' found in SpaCy under key 'PER' but not in Prompting
'Quiñones de Benavente' found in SpaCy under key 'PER' but not in Prompting
'Quiñones' found in SpaCy under key 'PER' but not in Prompting
'Antonio' found in SpaCy under key 'PER' but not in Prompting
'Osorio' found in SpaCy under key 'PER' but not in Prompting
'Infanta María Teresa de Austria' found in SpaCy under key 'PER' but not in Prompting
'Quiteria' found in SpaCy under key 'PER' but not in Prompting
'López Sustaete' found in SpaCy under key 'PER' but not in Prompting
'Manuela Bernarda 'la Grifona' found in SpaCy under key 'PER' but not in Prompting
'Séneca' found in SpaCy under key 'PER' but not in Prompting
'Jerónimo de?' found in SpaCy under key 'PER' but not

## 2. Preparing LLM for RE

In [7]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

In [9]:
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [16]:
with open("../data/introduccion.txt", "r", encoding="utf-8") as fr:
    text = fr.read()
    
with open("../data/historia_juan_rana.txt", "r", encoding="utf-8") as fr:
    text += fr.read()

### 2.1. First approach: Analyze the entire text passing NERs and text into the prompt

In [11]:
from langchain.prompts import PromptTemplate

entire_text_prompt = PromptTemplate(
    input_variables=["text", "entities"],
    template="""
        Dado el siguiente texto y una lista de entidades nombradas previamente extraídas, identifica todas las relaciones explícitas o implícitas entre dichas entidades.

        Las relaciones deben expresarse únicamente en el formato:
        (Entidad1)-[relación]-(Entidad2)

        Donde:

        - Entidad1 y Entidad2 deben coincidir con entidades de la lista proporcionada.

        - [relación] debe ser un verbo o una expresión verbal que indique la relación entre las entidades en el contexto del texto.

        - Si una relación puede expresarse con sinónimos más generales o normalizados (por ejemplo, "dirige", "es jefe de" → "dirige"), elige el término más general.

        - Ignora relaciones que no se puedan inferir directamente del texto.

        Lista de entidades (NERs):
        {entities}

        Texto de entrada:
        {text}

        Salida esperada:
        (EntidadA)-[relación]-(EntidadB)
        (EntidadC)-[relación]-(EntidadD)
        …
        
        Muestra la salida únicamente con las relaciones encontradas, sin ningún otro texto adicional.
    """,
)

In [17]:
response = llm.invoke(
    entire_text_prompt.invoke({
        "text": text,
        "entities": "\n".join(
            [
                f"{key}: {', '.join(values)}"
                for key, values in prompting_ners.items()
            ]
        ),
    })
).content

response

"(Cosme Pérez)-[fue conocido artísticamente como]-(Juan Rana)  \n(Cosme Pérez)-[fue hijo de]-(Damián Pérez)  \n(Cosme Pérez)-[fue hijo de]-(Isabel de Basto)  \n(Cosme Pérez)-[estuvo casado con]-(María de Acosta)  \n(Cosme Pérez)-[tuvo hija]-(Francisca María Pérez)  \n(Cosme Pérez)-[formaba parte de]-(compañía de Tomás Fernández)  \n(Cosme Pérez)-[formaba parte de]-(compañía de Juan Bautista Valenciano)  \n(Cosme Pérez)-[formaba parte de]-(compañía de Pedro de la Rosa)  \n(Cosme Pérez)-[formaba parte de]-(compañía de Antonio de Prado)  \n(Cosme Pérez)-[formaba parte de]-(compañía de Antonio García de Prado)  \n(Cosme Pérez)-[formaba parte de]-(compañía de Luis López Sustaete)  \n(Cosme Pérez)-[formó parte de]-(compañía de Diego Osorio)  \n(Cosme Pérez)-[formó parte de]-(compañía de Sebastián de Prado)  \n(Cosme Pérez)-[actuó en obra de]-(Luis de Belmonte)  \n(Cosme Pérez)-[actuó en obra de]-(Luis Quiñones de Benavente)  \n(Cosme Pérez)-[actuó en obra de]-(Antonio de Solís)  \n(Cosme Pér

In [18]:
relations = set()

for relation in response.split("\n"):
    print(relation.strip())
    relations.add(relation.strip())

(Cosme Pérez)-[fue conocido artísticamente como]-(Juan Rana)
(Cosme Pérez)-[fue hijo de]-(Damián Pérez)
(Cosme Pérez)-[fue hijo de]-(Isabel de Basto)
(Cosme Pérez)-[estuvo casado con]-(María de Acosta)
(Cosme Pérez)-[tuvo hija]-(Francisca María Pérez)
(Cosme Pérez)-[formaba parte de]-(compañía de Tomás Fernández)
(Cosme Pérez)-[formaba parte de]-(compañía de Juan Bautista Valenciano)
(Cosme Pérez)-[formaba parte de]-(compañía de Pedro de la Rosa)
(Cosme Pérez)-[formaba parte de]-(compañía de Antonio de Prado)
(Cosme Pérez)-[formaba parte de]-(compañía de Antonio García de Prado)
(Cosme Pérez)-[formaba parte de]-(compañía de Luis López Sustaete)
(Cosme Pérez)-[formó parte de]-(compañía de Diego Osorio)
(Cosme Pérez)-[formó parte de]-(compañía de Sebastián de Prado)
(Cosme Pérez)-[actuó en obra de]-(Luis de Belmonte)
(Cosme Pérez)-[actuó en obra de]-(Luis Quiñones de Benavente)
(Cosme Pérez)-[actuó en obra de]-(Antonio de Solís)
(Cosme Pérez)-[actuó en obra de]-(Agustín Moreto)
(Cosme Pé

In [22]:
print(f"Total de relaciones encontradas: {len(response.split("\n"))}, de las cuales {len(relations)} son únicas.")

Total de relaciones encontradas: 1669, de las cuales 63 son únicas.


In [29]:
for rel in relations:
    if len(rel.split("-")) >= 3:
        print(rel)

(Cosme Pérez)-[formaba parte de]-(compañía de Juan Bautista Valenciano)
(Cosme Pérez)-[actuó en obra de]-(Luis de Belmonte)
(Cosme Pérez)-[actuó en obra de]-(Antonio de Prado)
(Cosme Pérez)-[estuvo casado con]-(María de Acosta)
(Cosme Pérez)-[actuó en obra de]-(Juan Araña)
(Cosme Pérez)-[actuó en obra de]-(Luis Quiñones de Benavente)
(Cosme Pérez)-[actuó en obra de]-(Pedro de Cifuentes)
(Cosme Pérez)-[actuó en obra de]-(María de Quiñones)
(Cosme Pérez)-[actuó en obra de]-(Juan Ramos)
(Cosme Pérez)-[actuó en obra de]-(Juan Pérez de Montalbán)
(Cosme Pérez)-[fue hijo de]-(Damián Pérez)
(Cosme Pérez)-[actuó en obra de]-(Antonio de Solís)
(Cosme Pérez)-[formó parte de]-(compañía de Sebastián de Prado)
(Cosme Pérez)-[actuó en obra de]-(Mateo de Godoy)
(Cosme Pérez)-[actuó en obra de]-(Antonio Mejía)
(Cosme Pérez)-[actuó en obra de]-(Diego Osorio)
(Cosme Pérez)-[actuó en obra de]-(Antonio de Escamilla)
(Cosme Pérez)-[actuó en obra de]-(Agustín Moreto)
(Cosme Pérez)-[actuó en obra de]-(Juan V

In [30]:
with open("../out/full_text_re.txt", "w", encoding="utf-8") as fw:
    unique_relations = '\n'.join(rel for rel in relations if len(rel.strip().split("-")) >= 3)
    fw.write(unique_relations)

### 2.2. Second approach: Analyze each paragraph

In [31]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

In [32]:
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [33]:
with open("../data/introduccion.txt", "r", encoding="utf-8") as fr:
    text = fr.read()
    
with open("../data/historia_juan_rana.txt", "r", encoding="utf-8") as fr:
    text += fr.read()

In [34]:
from langchain.prompts import PromptTemplate

entire_text_prompt = PromptTemplate(
    input_variables=["text", "entities"],
    template="""
        Dado el siguiente texto y una lista de entidades nombradas previamente extraídas, identifica todas las relaciones explícitas o implícitas entre dichas entidades.

        Las relaciones deben expresarse únicamente en el formato:
        (Entidad1)-[relación]-(Entidad2)

        Donde:

        - Entidad1 y Entidad2 deben coincidir con entidades de la lista proporcionada.

        - [relación] debe ser un verbo o una expresión verbal que indique la relación entre las entidades en el contexto del texto.

        - Si una relación puede expresarse con sinónimos más generales o normalizados (por ejemplo, "dirige", "es jefe de" → "dirige"), elige el término más general.

        - Ignora relaciones que no se puedan inferir directamente del texto.

        Salida esperada:
        (EntidadA)-[relación]-(EntidadB)
        (EntidadC)-[relación]-(EntidadD)
        …
        
        Muestra la salida únicamente con las relaciones encontradas, sin ningún otro texto adicional.
        Lista de entidades (NERs):
        {entities}

        Texto de entrada:
        {text}
    """,
)

In [38]:
from tqdm import tqdm

entities = "\n".join([
    f"{key}: {', '.join(values)}"
    for key, values in prompting_ners.items()
])

total_re_found = 0
relations = set()

for paragraph in tqdm(text.split("\n\n")):
    if not paragraph.strip():
        continue

    response = llm.invoke(
        entire_text_prompt.invoke({
            "text": paragraph,
            "entities": entities,
        })
    ).content
    
    total_re_found += len(response.split("\n"))

    for relation in response.split("\n"):
        relation = relation.strip()
        if relation and len(relation.split("-")) >= 3:
            relations.add(relation)

100%|██████████| 37/37 [03:03<00:00,  4.96s/it]


In [40]:
print(f"Total de relaciones encontradas: {total_re_found}, de las cuales {len(relations)} son únicas.")

Total de relaciones encontradas: 497, de las cuales 485 son únicas.


In [41]:
with open("../out/per_paragraph_text_re.txt", "w", encoding="utf-8") as fw:
    unique_relations = '\n'.join(rel for rel in relations if len(rel.strip().split("-")) >= 3)
    fw.write(unique_relations)

### 2.3. Third approach: Analyze each sentence with overlap

In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [3]:
with open("../data/introduccion.txt", "r", encoding="utf-8") as fr:
    text = fr.read()
    
with open("../data/historia_juan_rana.txt", "r", encoding="utf-8") as fr:
    text += fr.read()

In [5]:
from langchain.prompts import PromptTemplate

entire_text_prompt = PromptTemplate(
    input_variables=["text", "context", "entities"],
    template="""
        Dada la siguiente frase y una lista de entidades nombradas previamente extraídas, identifica todas las relaciones explícitas o implícitas entre dichas entidades.
        Las frases pueden contener un contexto para ayudar a identificar las relaciones (para frases que comiencen con fue..., el cual..., etc.).

        Las relaciones deben expresarse únicamente en el formato:
        (Entidad1)-[relación]-(Entidad2)

        Donde:

        - Entidad1 y Entidad2 deben coincidir con entidades de la lista proporcionada.

        - [relación] debe ser un verbo o una expresión verbal que indique la relación entre las entidades en el contexto del texto.

        - Si una relación puede expresarse con sinónimos más generales o normalizados (por ejemplo, "dirige", "es jefe de" → "dirige"), elige el término más general.

        - Ignora relaciones que no se puedan inferir directamente del texto.

        Salida esperada:
        (EntidadA)-[relación]-(EntidadB)
        (EntidadC)-[relación]-(EntidadD)
        …
        
        Muestra la salida únicamente con las relaciones encontradas, sin ningún otro texto adicional.
        Lista de entidades (NERs):
        {entities}

        Contexto de la frase:
        {context}

        Frase de entrada:
        {text}
    """,
)

In [17]:
from tqdm.auto import tqdm
from collections import deque

entities = "\n".join([
    f"{key}: {', '.join(values)}"
    for key, values in prompting_ners.items()
])

sentences_of_context = 2 # Number of sentences to consider as context for each sentence
total_sentences = sum(len(paragraph.split(".")) for paragraph in text.split("\n\n") if paragraph.strip())

total_re_found = 0
relations = set()

for paragraph in tqdm(text.split("\n\n"), position=0):
    if not paragraph.strip():
        continue
        
    context = deque([])
    
    for sentence in tqdm(paragraph.split("."), position=1, leave=False):
        if not sentence.strip():
            continue

        response = llm.invoke(
            entire_text_prompt.invoke({
                "text": sentence.strip(),
                "context": "\n".join(context),
                "entities": entities,
            })
        ).content
    
        total_re_found += len(response.split("\n"))

        for relation in response.split("\n"):
            relation = relation.strip()
            if relation and len(relation.split("-")) >= 3:
                relations.add(relation)
        
        context.append(sentence.strip())
        if len(context) > sentences_of_context:
            context.popleft()

        

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/134 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
print(f"Total de relaciones encontradas: {total_re_found}, de las cuales {len(relations)} son únicas.")

Total de relaciones encontradas: 497, de las cuales 485 son únicas.


In [ ]:
with open("../out/per_paragraph_text_re.txt", "w", encoding="utf-8") as fw:
    unique_relations = '\n'.join(rel for rel in relations if len(rel.strip().split("-")) >= 3)
    fw.write(unique_relations)